---
### Ler o ficheiro de dados Chicago Crime - em csv usando o spark.read.csv

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, types as T
from pathlib import Path

data_dir = '../dados/'
file_chicago_crime_csv = data_dir + 'chicago_crime.csv'


---
### Apresentar o schema do csv

In [ ]:
spark = SparkSession.builder \
    .appName("ChicagoCrimeProject") \
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.memory", "8g")\
    .getOrCreate()

# Lê o CSV
df = spark.read.csv(file_chicago_crime_csv, header=True, inferSchema=True)

# Renomear colunas
for col in df.columns:
    novo_nome = col.strip().replace(" ", "_")
    df = df.withColumnRenamed(col, novo_nome)

# Mostra o esquema e primeiras linhas
df.printSchema()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: SparkSession$ does not exist in the JVM

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


### Profile Report usando ydata

In [4]:
from ydata_profiling import ProfileReport

profile_title = 'chicago_crime'

# aqui fazemos um sample mais pequeno pois não é possível com todo o dataset
sample_df = df.sample(0.1)

profile_report = ProfileReport(
    sample_df,
    title=profile_title,
    infer_dtypes=False,
    interactions=None,
    missing_diagrams=None,
    correlations={
        "auto": {"calculate": False},
        "pearson": {"calculate": False},
        "spearman": {"calculate": False},
    },
    html={"inline": False},  # Desativa a renderização inline de widgets
)

profile_report_file =  "../relatorios/profile_chicago_crime_before.html"
profile_report.to_file(Path(profile_report_file))
profile_report_file

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

'../relatorios/profile_chicago_crime_before.html'

### Cleanup dos dados


In [4]:
# assume raw DataFrame 'raw'
df_clean = (df
    # 1. cast de datas
    .withColumn("Date", F.to_timestamp("Date", "MM/dd/yyyy HH:mm:ss a"))
    .withColumn("Updated_On", F.to_timestamp("Updated_On", "MM/dd/yyyy HH:mm:ss a"))
    # 2. zeros -> null
    .withColumn("Ward", F.when(F.col("Ward") == 0, None).otherwise(F.col("Ward")).cast("int"))
    .withColumn("Community_Area", F.when(F.col("Community_Area") == 0, None)
                                   .otherwise(F.col("Community_Area")).cast("int"))
    .withColumn("X_Coordinate", F.when(F.col("X_Coordinate") == 0, None)
                                  .otherwise(F.col("X_Coordinate")))
    .withColumn("Y_Coordinate", F.when(F.col("Y_Coordinate") == 0, None)
                                  .otherwise(F.col("Y_Coordinate")))
    # 3. normalização de texto
    .withColumn("Block", F.upper(F.trim("Block")))
    .withColumn("Primary_Type", F.upper(F.trim("Primary_Type")))
    .withColumn("Description", F.upper(F.trim("Description")))
    .withColumn("Location_Description", F.upper(F.trim("Location_Description")))
    # 4. garantir tipos mais seguros
    .withColumn("ID", F.col("ID").cast(T.LongType()))  # evita overflow no futuro
    # 5. sanity-check de bounding box de Chicago
    .filter((F.col("Latitude").isNull()) |
            ((F.col("Latitude") > 41.6) & (F.col("Latitude") < 42.1) &
             (F.col("Longitude") < -87.5) & (F.col("Longitude") > -87.9)))
    # 6. remover duplicados
    .dropDuplicates()
)


In [ ]:
# Amostra de 10%
df_clean_small = df_clean.sample(fraction=0.1, seed=42)

# Guarda em Parquet
df_clean.coalesce(4).write.mode("overwrite").parquet("../dados/chicago_crime.parquet")

df_clean_small.coalesce(4).write.mode("overwrite").parquet("../dados/chicago_crime_small.parquet")